In [1]:
import polars as pl

In [4]:
games_df = pl.read_csv("games_df.csv")
games_df = games_df.filter(
    (pl.col("forbidMoving") == True)
    & (pl.col("forbidZooming") == False)
    & (pl.col("forbidRotating") == False)
    & (pl.col("roundCount") == 5)
    & (pl.col("roundTime") == 40)
    & (pl.col("mapSlug") == "world")
)
games_df.head(2)

game_id,forbidMoving,forbidZooming,forbidRotating,roundCount,mapSlug,roundTime,isSinglePlayer,scoringType,context,date
str,bool,bool,bool,i64,str,i64,bool,str,str,str
"""43b82fcf-29d8-4daf-84e9-80e029…",true,false,false,5,"""world""",40,false,"""Unknown""","""""","""2024-10-30T18:06:54.013Z"""
"""14cf63a7-1289-4b09-9ab8-75eb4f…",true,false,false,5,"""world""",40,false,"""Unknown""","""""","""2024-10-30T18:11:55.588Z"""


In [8]:
answers_df = pl.read_csv("answers_df.csv")
answers_df = answers_df.filter(
    pl.col("gameId").is_in(games_df["game_id"].unique())
)
answers_df.head(2)

gameId,roundNumber,lat,lng,country_code
str,i64,f64,f64,str
"""43b82fcf-29d8-4daf-84e9-80e029…",1,37.604549,126.933647,"""KR"""
"""43b82fcf-29d8-4daf-84e9-80e029…",2,55.926456,-4.126496,"""GB"""


In [6]:
guesses_df = pl.read_csv("guesses_df.csv")
guesses_df = guesses_df.filter(
    pl.col("gameId").is_in(games_df["game_id"].unique())
)
guesses_df.head(2)

roundNumber,lat,lng,distance,time,score,wasCorrect,gameId,playerName,code,playerId,payload,timestamp,lobby,countryGuess,coordinateGuess,battleRoyaleGameState,battleRoyalePlayer,duel,bullseye,player,country_code
i64,f64,f64,f64,i64,i64,bool,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
3,51.47018,5.786652,117653.422716,40,4621,false,"""ea8d34ee-d2c9-4283-834e-f0438f…","""moxxiq""","""LiveChallengeLeaderboardUpdate""",null,null,"""2024-08-21T18:47:34.5599291Z""",null,null,null,null,null,null,null,"""moxxiq""","""NL"""
3,-34.665525,-58.416663,3.9950e6,34,343,false,"""2d460ac2-f967-420b-9867-cefe08…","""Ishenko01y""","""LiveChallengeLeaderboardUpdate""",null,null,"""2024-12-11T18:39:45.8073149Z""",null,null,null,null,null,null,null,"""Ishenko01y""","""AR"""


In [11]:
cols = ["gameId", "roundNumber", "country_code"]
df = guesses_df[cols].join(answers_df[cols], on=["gameId", "roundNumber"], suffix="_answer")
df.head(2)

gameId,roundNumber,country_code,country_code_answer
str,i64,str,str
"""ea8d34ee-d2c9-4283-834e-f0438f…",3,"""NL""","""NL"""
"""2d460ac2-f967-420b-9867-cefe08…",3,"""AR""","""PE"""


In [89]:
df.filter(pl.col("country_code") != pl.col("country_code_answer")).with_columns(
    guess_answer=pl.when(pl.col("country_code") <= pl.col("country_code_answer"))
    .then(pl.col("country_code") + "_" + pl.col("country_code_answer"))
    .otherwise(pl.col("country_code_answer") + "_" + pl.col("country_code"))
)["guess_answer"].value_counts(sort=True)[:10]

guess_answer,count
str,u32
"""CA_US""",34
"""AT_DE""",31
"""GB_IE""",23
"""SZ_ZA""",20
"""ID_MY""",19
"""AU_US""",16
"""EE_FI""",16
"""EE_LT""",16
"""AU_NZ""",16


In [90]:
def get_correct(a):
    print(a[0], a[1])
    return a.filter(pl.col("country_code") == a[1][0]).shape[0]

tmp = df.with_columns(
    guessed=pl.col("country_code") == pl.col("country_code_answer")
).group_by("country_code_answer").agg(
    total=pl.col("country_code_answer").count(),
    guessed=pl.col("guessed").filter(pl.col("guessed") == True).count(),
    not_guessed=pl.col("guessed").filter(pl.col("guessed") == False).count()
).with_columns(
    ratio=pl.col("guessed") / pl.col("total")
)
tmp.filter(
    (pl.col("total") > 40)
    & (pl.col("ratio") < 0.2)
).sort(by="total", descending=True)

country_code_answer,total,guessed,not_guessed,ratio
str,u32,u32,u32,f64
"""AR""",78,13,65,0.166667
"""FI""",65,12,53,0.184615
"""ZA""",64,10,54,0.15625
"""BE""",61,10,51,0.163934
"""CL""",59,7,52,0.118644
"""UY""",58,9,49,0.155172
"""BM""",54,8,46,0.148148
"""PR""",51,7,44,0.137255
"""GT""",50,7,43,0.14


In [102]:
tmp.sort(by="ratio", descending=True)[:10]

country_code_answer,total,guessed,not_guessed,ratio
str,u32,u32,u32,f64
"""MC""",3,3,0,1.0
"""KR""",30,26,4,0.866667
"""UA""",35,28,7,0.8
"""GR""",30,23,7,0.766667
"""TN""",10,7,3,0.7
"""NL""",62,43,19,0.693548
"""IN""",28,18,10,0.642857
"""JP""",42,27,15,0.642857
"""US""",88,55,33,0.625
